In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

if 'spark' in locals() or 'spark' in globals():
    spark.stop()
    
spark = SparkSession\
    .builder\
    .appName("Testando")\
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/18 12:55:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [17]:
from pyspark.sql.functions import col, collect_list, concat_ws, current_date, date_add, date_format

In [2]:
dataDict = [
    (200001, "Michael", "Scott", "Regional Manager"),
    (200002, "Dwight", "Schrute", "Sales"),
    (200003, "Jim", "Halpert", "Sales"),
    (200004, "Phyllis", "Lapin", "Sales"),
    (200005, "Stanley", "Hudson", "Sales"),
    (200006, "Angela", "Martin", "Accounting"),
    (200007, "Kevin", "Malone", "Accounting"),
    (200008, "Oscar", "Martinez", "Accounting"),
    (200009, "Creed", "Bratton", "Quality Assurance"),
    (200010, "Meredith", "Palmer", "Supplier Relations"),
    (200011, "Pamela", "Beesly", "Recepctionist"),
    (200012, "Kelly", "Kapoor", "Customer Service"),
    (200013, "Ryan", "Howard", "Temp"),
    (200014, "Toby", "Flenderson", "Human Resources"),
    (200015, "Darryl", "Philbin", "Warehouse Foreman")
]

df1 = spark.createDataFrame(data = dataDict, schema = ["empId", "firstName", "lastName", "job"])
df1.show(5)

+------+---------+--------+----------------+
| empId|firstName|lastName|             job|
+------+---------+--------+----------------+
|200001|  Michael|   Scott|Regional Manager|
|200002|   Dwight| Schrute|           Sales|
|200003|      Jim| Halpert|           Sales|
|200004|  Phyllis|   Lapin|           Sales|
|200005|  Stanley|  Hudson|           Sales|
+------+---------+--------+----------------+
only showing top 5 rows



In [19]:
df1.withColumn('date', date_format(date_add(current_date(), -3), 'yyyyMMdd').cast('int') ).show()

+------+---------+----------+------------------+--------+
| empId|firstName|  lastName|               job|    date|
+------+---------+----------+------------------+--------+
|200001|  Michael|     Scott|  Regional Manager|20230515|
|200002|   Dwight|   Schrute|             Sales|20230515|
|200003|      Jim|   Halpert|             Sales|20230515|
|200004|  Phyllis|     Lapin|             Sales|20230515|
|200005|  Stanley|    Hudson|             Sales|20230515|
|200006|   Angela|    Martin|        Accounting|20230515|
|200007|    Kevin|    Malone|        Accounting|20230515|
|200008|    Oscar|  Martinez|        Accounting|20230515|
|200009|    Creed|   Bratton| Quality Assurance|20230515|
|200010| Meredith|    Palmer|Supplier Relations|20230515|
|200011|   Pamela|    Beesly|     Recepctionist|20230515|
|200012|    Kelly|    Kapoor|  Customer Service|20230515|
|200013|     Ryan|    Howard|              Temp|20230515|
|200014|     Toby|Flenderson|   Human Resources|20230515|
|200015|   Dar

In [11]:
df1.groupBy(col('job')).agg(collect_list(concat_ws('_', col('firstName'),col('lastName')))).show()

+------------------+-----------------------------------------------+
|               job|collect_list(concat_ws(_, firstName, lastName))|
+------------------+-----------------------------------------------+
|             Sales|                           [Jim_Halpert, Phy...|
|        Accounting|                           [Oscar_Martinez, ...|
| Quality Assurance|                                [Creed_Bratton]|
|              Temp|                                  [Ryan_Howard]|
|   Human Resources|                              [Toby_Flenderson]|
|  Regional Manager|                                [Michael_Scott]|
|Supplier Relations|                              [Meredith_Palmer]|
|     Recepctionist|                                [Pamela_Beesly]|
|  Customer Service|                                 [Kelly_Kapoor]|
| Warehouse Foreman|                               [Darryl_Philbin]|
+------------------+-----------------------------------------------+

